In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

data = pd.read_csv("/kaggle/input/upmarinedata/updatedMarineData.csv")

# forumla to convert heading into a unit vector
# (sine(heading), cos(heading))
# then you can multiply by speed for a more accurate picture of 
# what the vector is doing
import numpy as np
heading = 0
def HeadingToVector(heading):
    vectorX = np.sin(heading * np.pi / 180)
    vectorY = np.cos(heading * np.pi / 180)
    # Fix floating point errors
    if(vectorX < 0.00001 and vectorX > -0.00001):
        vectorX = 0
    if(vectorY < 0.00001 and vectorY > -0.00001):
        vectorY = 0
    return (vectorX, vectorY)

HeadingToVector(heading)

for index, row in data.iterrows():
    data.loc[index, "VelocityX"] = HeadingToVector(row["heading"])[0]
    data.loc[index, "VelocityY"] = HeadingToVector(row["heading"])[1]

data.to_csv('updatedMarineData.csv',index=False)
data.iloc[1000000:1000100]

In [ ]:
data = pd.read_csv("/kaggle/input/updatedmarinedataset/updatedMarineData.csv")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest

data.dropna(inplace=True)

X = data[["lat", "lon", "speed", "heading"]]
y = data[["timestamp", "mmsi"]]
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

one_class_svm = OneClassSVM(nu = 0.01, kernel = 'rbf', gamma = 'auto').fit(X_train) 

# Predict the anomalies 
prediction = one_class_svm.predict(X_test)

#Change the anomalies' values and to make it consistent with the true values
prediction = [1 if i==-1 else 0 for i in prediction]

y_test["Anomaly"] = 0
arrayIDs = [] # stores offending ship mmsi's
for i=0 in range(prediction.size):
    if(prediction[i] == 1):
        y_test.iloc[i, y_test.columns.get_loc("Anomaly")] = 1
        arrayIDs.append(y_test.iloc[i]["mmsi"]) # grabs mmsi

arrIDs = pd.DataFrame(arrayIDs)
print(arrIDs.size)
print(arrIDs.value_counts())

In [ ]:
# Isolation Forests take 2
IF = IsolationForest(contamination=0.01, n_estimators=100, n_jobs=-1, random_state=42).fit(X_train)

In [ ]:
predictions = IF.predict(X_test)
predictions = [1 if i==-1 else 0 for i in predictions]

In [ ]:
y_test["Anomaly"] = 0
arrayIDs = [] # stores offending ship mmsi's
for i in range(len(predictions)):
    if(predictions[i] == 1):
        y_test.iloc[i, y_test.columns.get_loc("Anomaly")] = 1
        arrayIDs.append(y_test.iloc[i]["mmsi"]) # grabs mmsi

In [ ]:
print(len(arrayIDs))
arrIDs = pd.DataFrame(arrayIDs)
print(arrIDs.size)
print(arrIDs.value_counts())

In [ ]:
anomaly = data.loc[data["mmsi"] == 230628000]
anomaly.head()

In [ ]:
import matplotlib.pyplot as plt

x = anomaly["lat"]
y = anomaly["lon"]
plt.scatter(x, y)
plt.show()

In [ ]:
# Create the box and whiskers (boxplot)
plt.boxplot(anomaly["speed"])
plt.show()

In [ ]:
# Create the box and whiskers (boxplot)
plt.boxplot(anomaly["heading"])
plt.show()

In [ ]:
y_test['scores']=IF.decision_function(X_test)
y_test.head()

In [ ]:
print(y_test["scores"].min(axis=0))

In [ ]:
import matplotlib.pyplot as plt

#create histogram
plt.hist(y_test["scores"], color='lightgreen', ec='black', bins=15)

In [ ]:
X_test.head()

In [ ]:
realtest = pd.DataFrame(np.array([[10, 200, -20, 120], [5, 20.857930, 28, 30], [57.802840, 20.857930, 4.48, 7.0], [59.770652, 24.816667, 7.46, 78.0]]), columns=['lat', 'lon', 'speed', 'heading'])
predictions = IF.predict(realtest)
predictions = [1 if i==-1 else 0 for i in predictions]
print(predictions)

In [ ]:
realtest